# Анализ временных свойств глаголов

Разбираем тексты с помощью Mystem

Выволдим количество глаголов непрошедшего, прошедшего и настоящего времени

Приводим полученные значения к векторному виду, сравниваем, делаем вывод

In [19]:
import re
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import string

import operator
from scipy.spatial import distance

import json

from collections import OrderedDict

In [11]:
def verb_tense_counter(mystemmed_line, tense_dict):
    """получаем на выход список слов, разобранных с помощью Mystem, ищем глаголы, создаем словарь """
    non_past = 0
    past = 0
    present = 0
    for i in mystemmed_line:
        keys = list(i.keys())
        values = list(i.values())
        word  = i['text']
        if ('analysis' not in keys):
            pass
        elif(i['analysis'] ==[]):
            pass
        else:
            grammar = i['analysis'][0]['gr']
            grammar_sep_by_comma = grammar.split(',')
            if (len(grammar_sep_by_comma) == 1):
                pass
            else:
                pos = re.match('[A-Z\s]+', grammar_sep_by_comma[0])[0]
                if(pos == "V"):
                    if ('непрош' in grammar):
                        tense_dict['non_past'] += 1
                    elif ('прош' in grammar):
                         tense_dict['past'] += 1
                    elif('наст' in grammar):
                         tense_dict['present'] += 1

In [13]:
def verb_tense_counter_file(file):
    m = Mystem()
    tense_dict = {'non_past':0,'past':0,'present':0}
    with open (file, "r", encoding = "utf-8") as file:
        for line in file.readlines():
            if(line != '\n' ):
                mystemmed_line = m.analyze(line)
                verb_tense_counter(mystemmed_line, tense_dict)
    return tense_dict

In [14]:
rzn_verb_tense_dict = verb_tense_counter_file('speech_rzn.txt')
yml_verb_tense_dict = verb_tense_counter_file('speech_yml.txt')
tula_verb_tense_dict = verb_tense_counter_file('speech_tula.txt')
tmn_verb_tense_dict = verb_tense_counter_file('speech_tmn.txt')
putin_verb_tense_dict = verb_tense_counter_file('speech_putin.txt')

In [16]:
def get_tense_vector(dictionary):
    """из полученных значений параметров глаголов создаем вектор"""
    vector = []
    tense_keys = ['non_past', 'past', 'present']
    for key in tense_keys:
        vector.append(dictionary[key])
    sum_vector = sum(vector)
    #делим 
    vector = [x/sum_vector for x in vector]
    n_vector = np.asarray(vector)
    return n_vector

In [17]:
rzn_verb_tense_vector = get_tense_vector(rzn_verb_tense_dict)
yml_verb_tense_vector = get_tense_vector(yml_verb_tense_dict)
tula_verb_tense_vector  = get_tense_vector(tula_verb_tense_dict)
tmn_verb_tense_vector  = get_tense_vector(tmn_verb_tense_dict)
putin_verb_tense_vector  = get_tense_vector(putin_verb_tense_dict)

In [ ]:
#ПОПАРНО БЕЗ ПОВТОРЕНИЙ СРАВНИВАЕМ ВСЕ ПОЛУЧЕННЫЕ ЗНАЧЕНИЯ

In [20]:
vectors_dict = {'ryazan':rzn_verb_tense_vector,"tyumen":tmn_verb_tense_vector,"tula": tula_verb_tense_vector, "yamal": yml_verb_tense_vector, "putin":putin_verb_tense_vector}
putin_cosine_distance_dict = {'ryazan':0,"tyumen":0,"tula": 0, "yamal": 0}
ordered_vectors_dict = OrderedDict(vectors_dict)
keys = list(ordered_vectors_dict.keys())
values = list(ordered_vectors_dict.values())

for dict_ind in range(len(keys)):
    current_vect = values[dict_ind]
    for compare_ind in range(dict_ind + 1,len(keys)):
        print(keys[dict_ind], "vs", keys[compare_ind])
        compared_vect = values[compare_ind]
        #print(current_vect,compared_vect)
        print(distance.cosine(current_vect,compared_vect))
        
        if(keys[compare_ind] == 'putin'):
            putin_cosine_distance_dict[keys[dict_ind]] = distance.cosine(current_vect,compared_vect)
    print()
sorted_x = sorted(putin_cosine_distance_dict.items(), key=operator.itemgetter(1))
print(sorted_x[-1][0],"is most similar to Putin speech")

ryazan vs tyumen
0.0032589601151555847
ryazan vs tula
0.0042693278635163034
ryazan vs yamal
0.002433909260803113
ryazan vs putin
0.0015354254094934738

tyumen vs tula
6.823582771064984e-05
tyumen vs yamal
6.018813223995778e-05
tyumen vs putin
0.0003208690677445336

tula vs yamal
0.0002565872951347803
tula vs putin
0.000684991816803926

yamal vs putin
0.00010312545242430193


ryazan is most similar to Putin speech
